<a href="https://colab.research.google.com/github/Ommy11/RAG-LLM/blob/main/Document_Analysis_using_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pdfplumber

pdf_path = "/content/google_terms_of_service_en_in.pdf"
output_text_file = "Extracted_text_data.pdf"

with pdfplumber.open(pdf_path) as pdf:
  extracted_data = ""
  for page in pdf.pages:
    extracted_data += page.extract_text()

with open(output_text_file, "w") as text_file:
  text_file.write(extracted_data)

print(f"Text extracted and saved to {output_text_file}")

Text extracted and saved to Extracted_text_data.pdf


In [11]:
## Now reading the pdf content
with open("/content/Extracted_text_data.pdf", "r") as file:
  document_text = file.read()

  print(document_text[:500]) ## preview first 500 characters


GOOGLE TERMS OF SERVICE
Effective May 22, 2024 | Archived versions
What’s covered in these terms
We know it’s tempting to skip these Terms of
Service, but it’s important to establish what you
can expect from us as you use Google services,
and what we expect from you.
These Terms of Service re ect the way Google’s business works, the laws that apply to
our company, and certain things we’ve always believed to be true. As a result, these Terms
of Service help de ne Google’s relationship with you as


In [12]:
## Now Summarize the document

from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the T5 model and tokenizer directly
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_text = "summarize: " + document_text[:1000]
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate the summary
summary_ids = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("Summary:", summary_text)

Loading weights:   0%|          | 0/131 [00:00<?, ?it/s]

Summary: GOOGLE TERMS OF SERVICE Effective May 22, 2024 Archived versions What’s covered in these terms We know it’s tempting to skip these Terms of Service, but it’s important to establish what you can expect from us as you use our services. these terms reect the way Google’s business works, the laws that apply to our company, and certain things we’ve always believed to be true.


In [13]:
# Now we split the documents into sentences and passages

import nltk
nltk.download('punkt')
nltk.download('punkt_tab') # Download punkt_tab resource
from nltk.tokenize import sent_tokenize

#split the text into sentences
sentences = sent_tokenize(document_text)

#combine sentences into passage
passages = []
current_passage = ""
for sentence in sentences:
  if len(current_passage.split()) + len(sentence.split()) < 200:
    current_passage += sentence
  else:
    passages.append(current_passage.strip())
    current_passage = sentence

if current_passage:
  passages.append(current_passage.strip())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [20]:
## Generate Questions from the Passages using LLMs
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the T5 question generation model and tokenizer directly
tokenizer_qg = T5Tokenizer.from_pretrained("valhalla/t5-base-qg-hl")
model_qg = T5ForConditionalGeneration.from_pretrained("valhalla/t5-base-qg-hl")

# Functions to generate questions
def generate_questions(passage, min_question=5):
  input_text = f"Generate Questions: {passage}"
  input_ids = tokenizer_qg.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

  # Generate the questions
  generated_ids = model_qg.generate(
      input_ids,
      max_length=150,
      num_beams=4,
      early_stopping=True,
      no_repeat_ngram_size=2 # Added to improve question quality
  )
  # Decode generated IDs
  decoded_questions = [tokenizer_qg.decode(g, skip_special_tokens=True) for g in generated_ids]

  # Process the decoded output, which can contain multiple questions separated by '<sep>'
  all_questions = []
  for q_block in decoded_questions:
      all_questions.extend([q.strip() for q in q_block.split('<sep>') if q.strip()])

  # Filter for questions and remove duplicates
  final_questions = [q for q in all_questions if q.endswith('?')]
  final_questions = list(dict.fromkeys(final_questions)) # Remove duplicates

  # If not enough questions are generated, try generating from smaller segments
  if len(final_questions) < min_question:
    passage_sentences = passage.split('. ')
    for i in range(len(passage_sentences)):
      if len(final_questions) >= min_question:
        break
      # Combine up to 2 sentences for additional input
      segment = " ".join(passage_sentences[i:i+2]).strip()
      if segment: # Ensure segment is not empty
          additional_input_text = "Generate Questions: " + segment
          additional_input_ids = tokenizer_qg.encode(additional_input_text, return_tensors="pt", max_length=512, truncation=True)
          additional_generated_ids = model_qg.generate(
              additional_input_ids,
              max_length=150,
              num_beams=4,
              early_stopping=True,
              no_repeat_ngram_size=2
          )
          additional_decoded_questions = [tokenizer_qg.decode(g, skip_special_tokens=True) for g in additional_generated_ids]
          for q_block_add in additional_decoded_questions:
              new_questions = [q.strip() for q in q_block_add.split('<sep>') if q.strip() and q.endswith('?')]
              final_questions.extend(new_questions)
              final_questions = list(dict.fromkeys(final_questions)) # Remove duplicates after adding new questions

  return final_questions[:min_question] # Return up to min_question

# generate questions from passages
for idx, passage in enumerate(passages):
    questions = generate_questions(passage) # Call the new function name
    print(f"Questions for Passage {idx + 1}: \n{passage}\n")
    print("Generated Questions : ")
    for q in questions:
      print(f"- {q}")
    print(f"\n{'-'*50}\n")

tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/260 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie shared.weight to encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Questions for Passage 1: 
GOOGLE TERMS OF SERVICE
Effective May 22, 2024 | Archived versions
What’s covered in these terms
We know it’s tempting to skip these Terms of
Service, but it’s important to establish what you
can expect from us as you use Google services,
and what we expect from you.These Terms of Service re ect the way Google’s business works, the laws that apply to
our company, and certain things we’ve always believed to be true.As a result, these Terms
of Service help de ne Google’s relationship with you as you interact with our services.For
example, these terms include the following topic headings:
What you can expect from us, which describes how we provide and develop our
services
What we expect from you, which establishes certain rules for using our services
Content in Google services, which describes the intellectual property rights to the
content you  nd in our services — whether that content belongs to you, Google, or
others
In case of problems or disagreements, which

In [24]:
# Answer the generated Qquestions uisng QA model

from transformers import pipeline

qa_Pipeline = pipeline("question-answering",model = "deepset/roberta-base-squad2")

def answer_unique_questions(passages_list, qa_Pipeline):
  answered_questions = set()

  for idx, passage in enumerate(passages_list):
    questions = generate_questions(passage)

    for question in questions:
      if question not in answered_questions:
        # Pass question and context as keyword arguments
        answer = qa_Pipeline(question=question, context=passage)
        print(f"Question: {question}")
        print(f"Answer: {answer['answer']}\n")
        answered_questions.add(question)
    print(f"{'='*50}\n")

answer_unique_questions(passages, qa_Pipeline)

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

RobertaForQuestionAnswering LOAD REPORT from: deepset/roberta-base-squad2
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Question: What does the Google Terms of Service cover?
Answer: certain things we’ve always believed to be true


Question: What is the name of Google's service provider?
Answer: Google LLC


Question: What do you expect from Google?
Answer: Provide a broad range of useful services


Question: What are some of the changes that Google may make to our services?
Answer: add or remove features and functionalities,
increase or decrease limits


Question: What do we expect from you?
Answer: Follow these terms and service-speci c additional terms


Question: What is Generative AI Prohibited Use Policy?
Answer: appropriate conduct that everyone using
those services must follow


Question: What are the rules?
Answer: to protect our services and users from abuse


Question: What are some of the ways you can use our services?
Answer: upload, submit, store, send, receive, or share
your content


Question: What type of content does Google not require a license for?
Answer: publicly-available factual


KeyboardInterrupt

